In [1]:
import pandas as pd
import numpy as np
import datetime
import json
import os
import itertools
import glob
import copy
import ray
import warnings


out_dir="./log/"
ray.init()


def replace_str(target):
    target = target.replace('\n', '')
    target = target.replace(',', '')
    return target

def find_value(arr, target, jumpto=1):
    try:
        num = replace_str(arr[arr.index(target)+jumpto])
    except:
        raise ValueError(
            f'{arr}, {target}')
    return num

with open('./sim_conf.json') as json_file:
    conf = json.load(json_file)


conf_share=ray.put(conf)
# TODO: fix directory finding as input
# origin_filename = "fsNprepNloadNtrain_Imagenet_default_resnet50_epoch5_b2048_worker12_thread0"
# origin_filename = "fsNprepNloadNtrain_Imagenet_randaugment_resnet50_epoch5_b2048_worker12_thread0"

origin_filename = conf["FILE"]
# origin_filename = "fsNprepNloadNtrain_openimage_default_resnet50_epoch1_b2048_worker12_thread0"

simp_filename = f'{origin_filename}_simp'
fetch_filename = f"{simp_filename}_fetchdifftime"
startdiff_filename = f"{simp_filename}_fetchstartdifftime"
parse_dir = "../dsanalyzer_parsed/DDP4GPUFULLTRACE/{suffix}/{filename}.csv"

perf_datafile = parse_dir.format(suffix="", filename= origin_filename)

simp_datafile = parse_dir.format(suffix="simp", filename= simp_filename)
fetch_datafile = parse_dir.format(suffix="simp", filename= fetch_filename)
startdiff_datafile = parse_dir.format(suffix="simp", filename= startdiff_filename)

perf_df = pd.read_csv(perf_datafile, index_col=0)
perf_df = perf_df[perf_df["Epoch"] == 1].sort_values(by=["Epoch", "Step"])
simp_df = pd.read_csv(simp_datafile, index_col=0)
fetch_df = pd.read_csv(fetch_datafile, index_col=0)
start_df = pd.read_csv(startdiff_datafile, index_col=0)

2021-09-01 20:35:47,775	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8265


In [2]:
# Train type and setup
info = perf_datafile.split('/')

trainType = find_value(info, "dsanalyzer_parsed")

if trainType.find("2GPU") != -1: 
    gpu_num = 2
if trainType.find("4GPU") != -1:
    gpu_num = 4
elif trainType.find("DDP") != -1:
    gpu_num = 8
else:
    gpu_num = 1
    
trainType_share = ray.put(trainType)

# Fetch time align

In [3]:
# Make gpu columns for simulation
gpu_start_col = []
gpu_fetch_col = []
gpu_fetch_done_col = []
gpu_training_stall_col = []
gpu_training_col = []
gpu_training_time_col = []
gpu_pure_training_col = []

single_gpu_prefetch_count = []
train_col= ["Epoch", "Index number"]

for i in range(gpu_num):
    gpu_start_col.append(f"Start time_gpu{i}")
    gpu_fetch_col.append(f"Fetch time (sec)_gpu{i}")
    gpu_fetch_done_col.append(f"Fetch done time (sec)_gpu{i}")
    gpu_training_stall_col.append(f"Training Stall time (sec)_gpu{i}")
    gpu_training_col.append(f"Training start time_gpu{i}")
    gpu_training_time_col.append(f"Iteration time (sec)_gpu{i}")
    gpu_pure_training_col.append(f"Pure training_gpu{i}")
    start_df[gpu_start_col[i]] = pd.to_datetime(
                                    start_df[gpu_start_col[i]], format='%Y-%m-%d %H:%M:%S.%f', errors='ignore')

# Init simulation informations
simulation_df = pd.DataFrame()

simulation_df[train_col] = fetch_df[train_col]

simulation_df = simulation_df.merge(right=fetch_df[train_col+gpu_fetch_col], 
                                    on = train_col)
simulation_df = simulation_df.merge(right=start_df[train_col+gpu_start_col], 
                                    on = train_col)

simulation_df["Min fetch time (sec)"] = fetch_df[gpu_fetch_col].min(axis=1)
simulation_df["Avg fetch time (sec)"] = fetch_df[gpu_fetch_col].mean(axis=1)
simulation_df["Min start time"] = start_df[gpu_start_col].min(axis=1)

simulation_df.sort_values(by=train_col, inplace = True)
simulation_df.reset_index(inplace = True, drop = True)
simulation_df = simulation_df[simulation_df["Epoch"] == 1]

for i in range(gpu_num):
    simulation_df[gpu_fetch_done_col[i]] = simulation_df[gpu_start_col[i]] + pd.to_timedelta(simulation_df[gpu_fetch_col[i]], 's')
    simulation_df[gpu_training_col[i]] = pd.to_datetime(
                                    perf_df[perf_df["GPU"] == i]["Time"].reset_index(drop=True), format='%Y-%m-%d %H:%M:%S.%f', errors='ignore')
    simulation_df[gpu_training_time_col[i]] = perf_df[perf_df["GPU"] == i]["Iteration time (sec)"].reset_index(drop=True)
    simulation_df[gpu_training_stall_col[i]] = perf_df[perf_df["GPU"] == i]["Training stall time (sec)"].reset_index(drop=True)
max_index_number = simulation_df["Index number"].max()
max_index_number_share = ray.put(max_index_number)

simulation_df_share = ray.put(simulation_df)

In [4]:
simulation_df

,Epoch,Index number,Fetch time (sec)_gpu0,Fetch time (sec)_gpu1,Fetch time (sec)_gpu2,Fetch time (sec)_gpu3,Start time_gpu0,Start time_gpu1,Start time_gpu2,Start time_gpu3,...,Iteration time (sec)_gpu1,Training Stall time (sec)_gpu1,Fetch done time (sec)_gpu2,Training start time_gpu2,Iteration time (sec)_gpu2,Training Stall time (sec)_gpu2,Fetch done time (sec)_gpu3,Training start time_gpu3,Iteration time (sec)_gpu3,Training Stall time (sec)_gpu3
0,1,0,8.112918,8.386826,8.087636,8.168932,2021-09-01 19:48:46.728082,2021-09-01 19:48:46.722174,2021-09-01 19:48:46.727364,2021-09-01 19:48:46.727068,...,9.777527,8.904674,2021-09-01 19:48:54.814999759,2021-09-01 19:48:55.268586,9.778004,8.673933,2021-09-01 19:48:54.895999750,2021-09-01 19:48:55.280864,9.777059,8.685712
1,1,1,8.426491,8.199841,8.215275,8.266806,2021-09-01 19:48:46.727509,2021-09-01 19:48:46.722159,2021-09-01 19:48:46.727725,2021-09-01 19:48:46.727194,...,0.895753,0.000002,2021-09-01 19:48:54.942999923,2021-09-01 19:48:56.095136,0.894281,0.000006,2021-09-01 19:48:54.994000042,2021-09-01 19:48:56.093251,0.895666,0.000002
2,1,2,8.220429,8.060581,11.568502,8.332884,2021-09-01 19:48:46.727571,2021-09-01 19:48:46.722419,2021-09-01 19:48:46.727498,2021-09-01 19:48:46.727116,...,2.217993,0.000002,2021-09-01 19:48:58.295999747,2021-09-01 19:48:58.612528,2.219373,1.345563,2021-09-01 19:48:55.060000074,2021-09-01 19:48:56.990001,2.218280,0.000002
3,1,3,8.567206,8.384353,8.628927,8.750336,2021-09-01 19:48:55.044794,2021-09-01 19:48:55.326647,2021-09-01 19:48:55.015073,2021-09-01 19:48:55.105664,...,5.680156,4.572775,2021-09-01 19:49:03.643999721,2021-09-01 19:49:03.968382,5.680703,4.482008,2021-09-01 19:49:03.855999751,2021-09-01 19:49:04.292639,5.680231,4.805935
4,1,4,8.976840,9.103020,8.669292,8.500594,2021-09-01 19:48:55.374160,2021-09-01 19:48:55.129980,2021-09-01 19:48:55.147708,2021-09-01 19:48:55.201406,...,0.894344,0.000002,2021-09-01 19:49:03.817000187,2021-09-01 19:49:04.660043,0.893107,0.000006,2021-09-01 19:49:03.702000102,2021-09-01 19:49:04.890191,0.894026,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,1,846,8.700867,8.742307,8.309392,8.876083,2021-09-01 20:32:38.071133,2021-09-01 20:32:35.288693,2021-09-01 20:32:42.396608,2021-09-01 20:32:35.873917,...,0.895935,0.000003,2021-09-01 20:32:50.705999790,2021-09-01 20:32:52.856106,0.895963,0.000006,2021-09-01 20:32:44.749999610,2021-09-01 20:32:52.857808,0.895931,0.000003
847,1,847,8.788954,8.310897,8.926671,8.654566,2021-09-01 20:32:36.299046,2021-09-01 20:32:42.242103,2021-09-01 20:32:36.710329,2021-09-01 20:32:36.001434,...,0.897361,0.000002,2021-09-01 20:32:45.636999536,2021-09-01 20:32:53.752485,0.898650,0.000006,2021-09-01 20:32:44.655999743,2021-09-01 20:32:53.753998,0.898609,0.000003
848,1,848,8.836856,7.349450,8.761064,8.692516,2021-09-01 20:32:39.316144,2021-09-01 20:32:52.088550,2021-09-01 20:32:37.389936,2021-09-01 20:32:36.898484,...,5.699361,4.828246,2021-09-01 20:32:46.151000104,2021-09-01 20:32:54.656551,5.698582,0.000006,2021-09-01 20:32:45.590999536,2021-09-01 20:32:54.650665,5.698188,0.000002
849,1,849,8.056848,8.511557,7.986414,8.299552,2021-09-01 20:32:46.992152,2021-09-01 20:32:44.249443,2021-09-01 20:32:50.900586,2021-09-01 20:32:44.983448,...,0.895900,0.000002,2021-09-01 20:32:58.886999697,2021-09-01 20:33:00.350174,0.895987,0.000006,2021-09-01 20:32:53.283000161,2021-09-01 20:33:00.348954,0.895796,0.000002


In [5]:
columns =  train_col + gpu_start_col + gpu_fetch_col + gpu_fetch_done_col

# Preprocessing time align
FETCH_ALIGNs = conf["FETCH_ALIGN"]
# Preprocessing start time align
START_ALIGNs = conf["START_ALIGN"]



def simulate_dataload(FETCH_ALIGN, START_ALIGN, idx, row, sim_dataload_df, sim_train_df, conf):
    simulated_gpu_info = {}
    
    for i in range(gpu_num):
        # Simulate based on the number of workers

        if START_ALIGN == "avg":
            next_fetch_start_time = row[gpu_start_col].max() - ((row[gpu_start_col].max() - row[gpu_start_col].min()) / 2)
        elif START_ALIGN == "min":
            next_fetch_start_time = row[gpu_start_col].min()
        else:
            next_fetch_start_time = row[gpu_start_col[i]]

        simulated_gpu_info[gpu_start_col[i]] = next_fetch_start_time
#         print(next_fetch_start_time)

        if FETCH_ALIGN == "avg":
            fetch_time = row["Avg fetch time (sec)"]
        elif FETCH_ALIGN == "min":
            fetch_time = row["Min fetch time (sec)"]
        else:
            fetch_time = row[gpu_fetch_col[i]]
            
        simulated_gpu_info[gpu_fetch_col[i]] = fetch_time # + conf["FETCH_DELAY"]
                                                                                                                 
        simulated_gpu_info[gpu_fetch_done_col[i]] = simulated_gpu_info[gpu_start_col[i]] + pd.to_timedelta(simulated_gpu_info[gpu_fetch_col[i]], 's')
    return simulated_gpu_info

def simulate_training(FETCH_ALIGN, START_ALIGN, idx, row, sim_dataload_df, sim_train_df):
    # FIXME: Manually tune with idx
    fetch_done_times = sim_dataload_df.loc[idx][gpu_fetch_done_col]
    
    slowest_fetch_time = fetch_done_times.max()
    
    # INFO
    info_fetch_done_times = row[gpu_fetch_done_col]
    info_slowest_fetch_time = info_fetch_done_times.max()
    info_training_start_times = row[gpu_training_col].values
    info_training_times = row[gpu_training_time_col].values
    info_training_stall_times = row[gpu_training_stall_col].values
    info_pure_training_times = info_training_times - info_training_stall_times.max()
    info_training_end_times = info_training_start_times + pd.to_timedelta(info_pure_training_times, 's')
    info_data_require_times = info_training_start_times - pd.to_timedelta(info_training_stall_times.max(), 's')
    
    if FETCH_ALIGN or START_ALIGN:
        sim_gaps = np.array([])
        training_stall_times = np.array([])
        for _info_fetch_done_time, _fetch_done_time, _training_stall_time in zip(info_fetch_done_times.values, fetch_done_times.values, info_training_stall_times):
            if _info_fetch_done_time > _fetch_done_time:
                _sim_gap = _info_fetch_done_time - _fetch_done_time
                new_training_stall_time = _training_stall_time - (_sim_gap / np.timedelta64(1, 's'))
                if new_training_stall_time < 0.0:
                    new_training_stall_time = 0.000002
            elif _info_fetch_done_time < _fetch_done_time:
                _sim_gap = _fetch_done_time - _info_fetch_done_time
                # FIXME: Need to check it actually slowdown
                if _training_stall_time > 0.01:
                    new_training_stall_time = _training_stall_time + (_sim_gap / np.timedelta64(1, 's'))
                else:
                    new_training_stall_time = _training_stall_time
            else:
                _sim_gap = pd.Timedelta(0, 's')
                new_training_stall_time = _training_stall_time
#             print(sim_gaps, type(_sim_gap))
            sim_gaps = np.append(sim_gaps, _sim_gap)
            training_stall_times = np.append(training_stall_times, new_training_stall_time)
        training_times = training_stall_times.max() + info_pure_training_times
        training_end_times = info_training_start_times + pd.Timedelta(training_times.max())
    else:
        training_times = info_training_times
        training_end_times = info_training_end_times
        training_stall_times = info_training_stall_times
        
    done_row = {"Slowest end time":slowest_fetch_time, "Training end time":training_end_times.max(), "Training time": training_times.max()}

    for i in range(gpu_num):
        done_row[gpu_training_col[i]] = info_training_start_times[i]
        done_row[gpu_pure_training_col[i]] = info_pure_training_times[i]
        done_row[gpu_training_stall_col[i]] = training_stall_times[i]
        
    delay_time = training_stall_times.max() - training_stall_times.min()
    done_row["Delay time (sec)"] = delay_time
    done_row["Training stall time (sec)"] = training_stall_times.max()
    
    return done_row

@ray.remote
def simulation(FETCH_ALIGN,START_ALIGN, conf, trainType, simulation_df, max_index_number):
    warnings.filterwarnings("ignore")
    simulated_df = pd.DataFrame(columns=columns)
    done_df = pd.DataFrame(columns=["Slowest end time", "Training time","Training end time", "Delay time (sec)","Training stall time (sec)"]+gpu_training_col+gpu_pure_training_col+gpu_training_stall_col)
    
    for idx, row in simulation_df.iterrows():
        simulated_row = {}
        simulated_row[train_col[0]] = row[train_col[0]]
        simulated_row[train_col[1]] = row[train_col[1]]
        
        simulated_row.update(simulate_dataload(FETCH_ALIGN, START_ALIGN, idx, row, simulated_df, done_df, conf))

        simulated_df.loc[len(simulated_df)] = simulated_row
#         print(done_row)
        done_df.loc[len(simulated_df)] = simulate_training(FETCH_ALIGN, START_ALIGN, idx, row, simulated_df, done_df)
#     print(done_df)   
    sim_train_df = pd.DataFrame()
    sim_train_df[train_col] = simulation_df[train_col].reset_index(drop=True)
    sim_train_df["Iteration time (sec)"] = done_df["Training time"].reset_index(drop=True)
    sim_train_df[gpu_training_stall_col] = done_df[gpu_training_stall_col].reset_index(drop=True)
    sim_train_df["Training stall time (sec)"] = done_df["Training stall time (sec)"].reset_index(drop=True)
    sim_train_df["Delay time (sec)"] = done_df["Delay time (sec)"].reset_index(drop=True)
    sim_train_df["Batch fetch time diff (sec)"]  = simulated_df[gpu_fetch_col].max(axis=1) - simulated_df[gpu_fetch_col].min(axis=1)
    sim_train_df["Batch fetch start time diff (sec)"] = (simulated_df[gpu_start_col].max(axis=1) - simulated_df[gpu_start_col].min(axis=1)).dt.total_seconds()
    
    os.makedirs(f"./no_deps/{trainType}/FETCH_ALIGN_{FETCH_ALIGN}_START_ALIGN_{START_ALIGN}/debug", exist_ok=True)
    
    sim_train_df.to_csv(f"./no_deps/{trainType}/FETCH_ALIGN_{FETCH_ALIGN}_START_ALIGN_{START_ALIGN}/{origin_filename}.csv")
    
    simulated_df.to_csv(f"./no_deps/{trainType}/FETCH_ALIGN_{FETCH_ALIGN}_START_ALIGN_{START_ALIGN}/debug/dataload_sim.csv")
    done_df.to_csv(f"./no_deps/{trainType}/FETCH_ALIGN_{FETCH_ALIGN}_START_ALIGN_{START_ALIGN}/debug/train_sim.csv")
    simulation_df.to_csv(f"./no_deps/{trainType}/FETCH_ALIGN_{FETCH_ALIGN}_START_ALIGN_{START_ALIGN}/debug/sim_info.csv")
    return simulated_df

# Initialization
conf_combinations = list(itertools.product(FETCH_ALIGNs, START_ALIGNs))
result_ids = []
for FETCH_ALIGN, START_ALIGN in conf_combinations:
    result_ids.append(simulation.remote(FETCH_ALIGN,START_ALIGN, conf_share, 
                                        trainType_share, simulation_df_share, max_index_number_share))
    #     # Debug break
#     break
    
while len(result_ids):
    done_id, result_ids = ray.wait(result_ids)
    ray.get(done_id)


In [6]:
dir_names = glob.glob(
               f"./no_deps/{trainType}/*/*.csv")

dataset_col_name = ["Align","Log type", "Dataset size(GB)", "Model", "Augmentation", "Worker", "Worker batch size", "Epoch", "Batch size", "Avg iteration time (sec)",
                               "Avg training stall time (sec)", "Avg delay time (sec)", "Effective Delay time (%)", "Avg preprocessing start diff (sec)", "Avg preprocessing diff (sec)"]
total_log = [] 

for logdir in dir_names:
    split_logdir = logdir.split('/')
    align_type = split_logdir[-2]
    parse_filename = split_logdir[-1]
    logdir_list = parse_filename.split("_")
#         print(logdir_list)
    logtype = logdir_list[0]
    dataset = logdir_list[1]
    aug = find_value(logdir_list, dataset)
    model = find_value(logdir_list, aug)
    epoch = find_value(logdir_list, model)
    batchsize = find_value(logdir_list, epoch)
    worker = find_value(logdir_list, batchsize)
    thread = find_value(logdir_list, worker).replace(".csv","")
    model = model.replace("_", "")

    epoch_num = int(epoch.replace("epoch", ""))
    batchsize_num = int(batchsize.replace("b", ""))
    single_batchsize_num = batchsize_num/gpu_num
    conf["WORKER_NUM"] = int(worker.replace("worker", ""))
    thread_num = int(thread.replace("thread", ""))
    
    df = pd.read_csv(logdir, index_col=None)
    df = df[df["Index number"]>10]
#     gpu_log = []

#     for i in range(gpu_num):
#         gpu_df = df[df["GPU"] == i].dropna()
#         gpu_epoch_time = f'{round(gpu_df["Iteration time (sec)"].mean(),4)}±{round(gpu_df["Iteration time (sec)"].std(),4)}'
#         gpu_log.append(gpu_epoch_time)
#         gpu_stall_time = f'{round(gpu_df["Training stall time (sec)"].mean(),4)}±{round(gpu_df["Training stall time (sec)"].std(),4)}'
#         gpu_log.append(gpu_stall_time)


#     df = df[df["Step"] > 10]
#     filtered_avg_epoch_time = round(
#         df["Iteration time (sec)"].astype(float).sum()/(int(epoch_num)-1), 2)
#     print(df.describe())
#     print("\n")
    # print(parse_filename, df["Iteration time (sec)"])
    iter_origin = df["Iteration time (sec)"].mean()
    iter_avg = f'{round(iter_origin,4)}±{round(df["Iteration time (sec)"].std(),4)}'
    data_avg = f'{round(df["Training stall time (sec)"].mean(),4)}±{round(df["Training stall time (sec)"].std(),4)}'
    delay_avg = f'{round(df["Delay time (sec)"].mean(),4)}±{round(df["Delay time (sec)"].std(),4)}'
    fetch_avg = f'{round(df["Batch fetch time diff (sec)"].mean(),4)}±{round(df["Batch fetch time diff (sec)"].std(),4)}'
    fetch_start_avg = f'{round(df["Batch fetch start time diff (sec)"].mean(),4)}±{round(df["Batch fetch start time diff (sec)"].std(),4)}'
    
    # NOTE: fixed effective delay time (0.030)
    effective_delay_percent = float(len(df[df["Delay time (sec)"] > 0.030])) / float(len(df)) * 100.0
#     throughput_origin = single_batchsize_num/iter_origin
#     throughput_avg = round(throughput_origin, 4)
    # # [Note] :
    # # Deprecated throughput avg,
    # # theoretically wrong in mathematic, Please check below link
    # # https://fxloader.com/inverse_of_an_average_compared_to_averages_of_inverses/
    # throughput_avg = f'{round(df["Throughput (image/sec)"].mean(),2)}±{round(df["Throughput (image/sec)"].std(),4)}'

    # FIXME: Hard coded as imagenet avg size (MB)
#     processed_data_avg = throughput_origin * 105.53 / 1024

    total_loglet = [align_type, logtype, dataset, model, aug, conf["WORKER_NUM"], thread_num, epoch_num, batchsize_num,
                    iter_avg, data_avg, delay_avg, effective_delay_percent, fetch_start_avg, fetch_avg]
#     total_loglet.extend(gpu_log)
    total_log.append(total_loglet)

avg_df = pd.DataFrame(total_log,
                      columns=dataset_col_name)
avg_df.dropna().to_csv(f"./no_deps/{trainType}/total_summary.csv",
                       sep=',', na_rep='NA')

2021-09-01 20:59:44,153	ERROR worker.py:468 -- print_logs: Connection closed by server.
2021-09-01 20:59:44,154	ERROR worker.py:1191 -- listen_error_messages_raylet: Connection closed by server.
2021-09-01 20:59:44,154	ERROR import_thread.py:88 -- ImportThread: Connection closed by server.
